In [ ]:
#check GPU
!nvidia-smi

In [ ]:
from __future__ import absolute_import, division, print_function,unicode_literals

try:
    !pip uninstall tb-nightly tesorboardX tensorboard
    !pip install tf-nightly
except Exception:
    pass





In [2]:
import tensorflow as tf

import os
import datetime
import tensorflow_datasets as tfds

In [ ]:
%load_ext tensorboard

In [ ]:
import pkg_resources

for entry_point in pkg_resources.iter_entry_points('tensorboard_plugins'):
    print(entry_point.dist)

In [3]:
print(tf.__version__)

2.4.0


In [4]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
dataset, info=tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', with_info=True)

Shuffling and writing examples to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0.incompletePYRHPN/amazon_us_reviews-train.tfrecord


Dataset amazon_us_reviews downloaded and prepared to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0. Subsequent calls will reuse this data.


In [6]:
train_dataset=dataset['train']

In [ ]:
info

In [ ]:
print(train_dataset)

In [ ]:
len(list(train_dataset))

In [7]:
BUFFER_SIZE=30000
BATCH_SIZE=128

In [8]:
train_dataset=train_dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)

In [ ]:
for reviews in train_dataset.take(2):
    print(reviews)

In [ ]:
for reviews in train_dataset.take(10):
    review_text=reviews['data']
    print(review_text.get('review_body').numpy())
    print(review_text.get('star_rating'))
    print(tf.where(review_text.get('star_rating')>3,1,0).numpy())

In [ ]:
tokenizer=tfds.features.text.Tokenizer()

In [ ]:
vocabulary_set=set()
for _, reviews in train_dataset.enumerate():
    #print(reviews)
    review_text=reviews['data']
    reveiws_tokens=tokenizer.tokenize(review_text.get('review_body').numpy())
    vocabulary_set.update(reviews_tokens)

In [ ]:
vocab_size=len(vocabulary_set)
vocab_size

In [ ]:
encoder=tfds.features.text.TokenTextEncoder(vocabulary_set)

In [ ]:
print(vocabulary_set)

In [ ]:
or reviews in train_dataset.take(5):
    review_text=reviews['data']
    print(review_text.get('review_body').numpy())
    encoded_example=encoder.encode(review_text.get('review_body').numpy())
    print(encoded_example)

In [ ]:
for index in encoded_example:
    print('{}------>{}'.format(index, encoder.decode([index])))

In [ ]:
def encode(text_tensor, label_tensor):
    encoded_text=encoder.encode(text_tensor.numpy())
    label=tf.where(label_tensor>3,1,0)
    return encoded_text, label

In [ ]:
def encode_map_fn(tensor):
    text=tensor['data'].get('review_body')
    label=tensor['data'].get('star_rating')

    encoded_text, label=tf.py_function(encode,
                                       inp=[text,label],
                                       Tout=(tf.int64, tf.int32))
    
    encoded_text.set_shape([None])
    label.set_shape([])

    return encoded_text, label
    


In [ ]:
ar_encoded_data=train.dataset.map(encode_map_fn)

In [ ]:
for f0,f1 in ar_encoded_data.take(2):
    print(f0)
    print(f1)

In [ ]:
TAKE_SIZE=10000

train_data=ar_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data=train_data.padded_batch(BATCH_SIZE)

test_data=ar_encoded_data.take(TAKE_SIZE)
test_data=test_data.padded_batch(BATCH_SIZE)


In [ ]:
vocab_size +=1

In [ ]:
sample_text, sample_labels=next(iter(test_data))

sample_text[0], sample_labels[0]

In [ ]:
for f0,f1 in test_data.take(2):
    print(tf.unique_with_counts(f1)[2].numpy())

In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size,128))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)))
#model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
for units in [64,64]:
    model.add(tf.keras.layers.Dense(units, activation='relu'))
    #model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(1))

In [ ]:
!rm -r /tmp/logs/

In [ ]:
logdir=os.path.join("/tmp/logs/", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback=tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
checkpointer=tf.keras.callbacks.ModelCheckpoint(filepath='/tmp/final_sentiment_analysis.hdf5', verbose=1, save_best_only=True))

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history=model.fit(train_data, epochs=4, validation_data=test_data, callbacks=[tensorboard_callback,checkpointer])

In [ ]:
model.save('/tmp/final_sentiment_analysis.hdf5')

In [ ]:
!ls -alrt /tmp/*.hdf5

In [ ]:
eval_loss, eval_acc=model.evaluator(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

In [ ]:
for f0,f1 in test_data.take(1):
    print(f1)
    print(model.predict(f0))


In [ ]:
model.layers

In [ ]:
model.summary()

In [ ]:
model.get_layer('embedding').output

In [ ]:
!nvidia-smi

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_' + metric],'')
